In [25]:
# This notebook conglomerates our different data into one csv with the cities of interest.
import pandas as pd
import numpy as np

In [26]:
df_education = pd.read_csv("shared_data_read_only/MKE_Fellows/Education_Data.csv", low_memory=False)

In [27]:
df_census = pd.read_csv("shared_data_read_only/MKE_Fellows/Census_Data.csv", low_memory=False)

In [28]:
df_salary = pd.read_csv("shared_data_read_only/MKE_Fellows/Career_Outcomes_Data.csv", low_memory=False)

In [32]:
# Create Geographic Area Name column for nonprofits df
df_nonprofits = pd.read_csv("test/Team-13/raw_data/educational_services_counties_sorted.csv", low_memory=False)

df_nonprofits["fipstate_mapped"] = df_nonprofits["fipstate_mapped"].str.capitalize()

df_nonprofits["Geographic Area Name"] = df_nonprofits["county"] + ", " + df_nonprofits["fipstate_mapped"]

df_nonprofits = df_nonprofits.drop(columns=['fipstate_mapped', 'county'])

In [33]:
df_internship_listings = pd.read_csv("test/Team-13/raw_data/internships_by_county.csv")

In [34]:
column_names = df_education.iloc[0]
df_education.columns = column_names

In [35]:
column_names = df_census.iloc[0]
df_census.columns = column_names

In [36]:
# High school grad or higher rate in black males
grad_column_name = 'Estimate!!Percent Male!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Black alone!!High school graduate or higher'
df_education[grad_column_name] = pd.to_numeric(df_education[grad_column_name], errors='coerce')

# Bachelors degree or higher rate in black males
college_grad_column_name = "Estimate!!Percent Male!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Black alone!!Bachelor's degree or higher"
df_education[college_grad_column_name] = pd.to_numeric(df_education[college_grad_column_name], errors='coerce')


In [37]:
# Num black households
household_column_name = 'Estimate!!Total!!Households!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Black or African American alone'
df_census[household_column_name] = pd.to_numeric(df_census[household_column_name], errors='coerce')

In [38]:
geographic_area_column = 'Geographic Area Name'


In [39]:
column_names = df_salary.iloc[0]
df_salary.columns = column_names

In [40]:
# Total household earnings
income_column_name = 'Estimate!!INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Total households!!With earnings!!Mean earnings (dollars)'

df_salary[income_column_name] = pd.to_numeric(df_salary[income_column_name], errors='coerce')

median_income = df_salary.groupby(geographic_area_column)[income_column_name].median().reset_index()

median_income.columns = [geographic_area_column, 'Median Income']

In [41]:
# Merge our data sets using Geographic Area Name as an identiifier
df_merged = pd.merge(df_education, df_census, left_on=geographic_area_column, right_on=geographic_area_column, how='inner')
df_merged = pd.merge(df_merged, df_salary, left_on=geographic_area_column, right_on=geographic_area_column, how='inner')


In [42]:
# Remove unneeded columns
columns_to_keep = [geographic_area_column, grad_column_name, college_grad_column_name, income_column_name, household_column_name]
new_df = df_merged[columns_to_keep]

# Rename the columns for clarity
new_column_names = {
    geographic_area_column: 'Geographic Area Name',
    grad_column_name: 'High School Graduate or Higher % for Black Males',
    college_grad_column_name: "Bachelor's Degree or Higher % for Black Males",
    income_column_name: 'Median Total Household Earnings',
    household_column_name: 'Total Households for Black Americans'
}

new_df = new_df.rename(columns=new_column_names)
new_df = new_df.iloc[1:].reset_index(drop=True)


In [43]:
# Counties have different "tracts", combine all of these tracts into one county entry.
new_df['County'] = new_df['Geographic Area Name'].apply(lambda x: x.split(';')[1].strip())
new_df['State'] = new_df['Geographic Area Name'].apply(lambda x: x.split(';')[2].strip())

def weighted_avg(df, val_col, weight_col):
    total_weight = df[weight_col].sum()
    if total_weight == 0:  # Check if total weight is zero
        return 0  # Return 0 or an appropriate value for this case
    return (df[val_col] * df[weight_col]).sum() / total_weight

df_combined = new_df.groupby(['County', 'State']).agg({
    'Median Total Household Earnings': 'mean',
    'Total Households for Black Americans': 'sum'
}).reset_index()

# Average out the tracts into a single county entry for High School Grad Rate
df_combined['High School Graduate or Higher % for Black Males'] = new_df.groupby(['County', 'State']).apply(
    lambda x: weighted_avg(x, 'High School Graduate or Higher % for Black Males', 'Total Households for Black Americans')
).reset_index(drop=True)

# Average out the tracts into a single county entry for Bachelor's Degree Grad Rate
df_combined["Bachelor's Degree or Higher % for Black Males"] = new_df.groupby(['County', 'State']).apply(
    lambda x: weighted_avg(x, "Bachelor's Degree or Higher % for Black Males", 'Total Households for Black Americans')
).reset_index(drop=True)

df_combined.rename(columns={'County': 'Geographic Area Name'}, inplace=True)

df_combined['Geographic Area Name'] = df_combined['Geographic Area Name'] + ', ' + df_combined['State']

df_combined.drop(columns='State', inplace=True)

# List of Counties we will consider in our final dataset
desired_counties = ['Harris County, Texas', 'Cook County, Illinois', "Los Angeles County, California", "Orange County, California", "Philadelphia County, Pennsylvania", "King County, Washington", "Kings County, California"]  # Add your desired county names here

# Filter to include only the desired counties
df_filtered = df_combined[df_combined['Geographic Area Name'].isin(desired_counties)]


df_filtered = pd.merge(df_filtered, df_nonprofits, left_on=geographic_area_column, right_on=geographic_area_column, how='inner')
df_filtered = pd.merge(df_filtered, df_internship_listings, left_on=geographic_area_column, right_on=geographic_area_column, how='left')


In [45]:
df_filtered.to_csv('test/Team-13/data_processing/county_data.csv', index=False, na_rep='N/A')
print(df_filtered)

                Geographic Area Name  Median Total Household Earnings  \
0              Cook County, Illinois                    111894.560241   
1               Harris County, Texas                    100314.270758   
2            King County, Washington                    162934.105263   
3           Kings County, California                     81824.241379   
4     Los Angeles County, California                    117089.302600   
5          Orange County, California                    146437.659574   
6  Philadelphia County, Pennsylvania                     89702.958656   

   Total Households for Black Americans  \
0                              480816.0   
1                              351926.0   
2                               54765.0   
3                                2554.0   
4                              311590.0   
5                               19013.0   
6                              261888.0   

   High School Graduate or Higher % for Black Males  \
0              